In [1]:
from binance.spot import Spot 
from sys import stderr
import sys, os,datetime,requests,json,pandas as pd,numpy as np,matplotlib.pyplot as plt
import time,math, gc

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import sklearn
from sklearn.model_selection import train_test_split
min_max_scaler = sklearn.preprocessing.MinMaxScaler()



import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def import_api(time_last,symbol="BTCUSDT",limit=12*60):
    params={"symbol":symbol,"limit":limit,"interval":"1m","endTime":time_last,"startTime":(time_last-12*3600*1000)}
    r=requests.get(url="https://api.binance.com/api/v3/klines", params=params)
    df=pd.DataFrame(r.json())
    return df

def transform_df (df):
    column_names=['Open_time','Open','High','Low','Close','Volume','Close_time','Quote_asset_volume',
               'Number_of_trades','Taker_buy_base_asset_volume','Taker_buy_quote_asset_volume','Ignore']
    df=df.reset_index(drop=True)
    df.set_axis(column_names,axis=1,inplace=True)
    for col in df.columns:
        df[col] = df[col].astype(float)
    weekday=[]
    month=[]
    for i in range(df['Open'].size):
        dt_open=datetime.datetime.fromtimestamp(df['Open_time'][i]//1000)
        dt_close=datetime.datetime.fromtimestamp(df['Close_time'][i]//1000)
        df['Open_time'][i]=dt_open.hour*3600+dt_open.minute*60+dt_open.second
        df['Close_time'][i]=dt_close.hour*3600+dt_close.minute*60+dt_close.second
        weekday.append(dt_open.weekday())
        month.append(dt_open.month)
        #df['Open_time'][i]=dt_open.strftime("%I:%M:%S")
        #df['Close_time'][i]=dt_close.strftime("%I:%M:%S")
    df['weekday']=weekday
    df['month']=month
    df=df.drop(df.columns[[7,10,11,13]],axis=1)
    del(month)
    del(weekday)
    return df


def moving_average(data,range_,concat=60):
    leftover=len(data)%concat
    data=data[::concat].reset_index(drop=True)
    mean=[]
    arr=[]
    for i in range(range_):
        this_mean=0
        for j in range(i+1):
            this_mean+=data[j]
        mean.append(this_mean/(j+1))
    if leftover:
        data_size=len(data)-1
    else:
        data_size=len(data)
    for i in range(range_,data_size):
        this_mean=data[i]
        for j in range(1,range_):
            this_mean+=data[i-j]
        mean.append(this_mean/(range_))
    arr=leftover*[mean[0]]
    for i in range(len(mean)):
        arr+=concat*[mean[i]]
    
    
    return arr

def generate_previous(df,count,col_name):
    #idxes=
    df=df.reset_index(drop=True)
    values=list(df[col_name][0:count])
    columns={}
    for i in range(count):
        this_name="prev_"+col_name+"_"+str(i+1)
        columns[this_name]=[]
    for i in range(count,df[col_name].size):
        for j in range(count):
            this_name="prev_"+col_name+"_"+str(j+1)
            columns[this_name].append(values[-(j+1)])
        values.pop(0)
        values.append(df[col_name][i])
    size=df[col_name].size
    df=df[count:size]
    for i in range(count):
        this_name="prev_"+col_name+"_"+str(i+1)
        df[this_name]=columns[this_name]
    
    return df.reset_index(drop=True)

def df_to_X_y(df, window_size=5):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [a for a in (df.iloc[i:i+window_size].drop("target",axis=1).values)]
        X.append(row)
        label = df["target"][i+window_size-1]
        y.append(label)
    return np.array(X), np.array(y)


In [4]:
df=pd.read_csv(r'G:\eth_usdt_5min.csv')[20000:700000].reset_index(drop=True)
target=[]
window=3
for i in range(df["Open"].size-window):
    target.append(df["Open"][i+window-1])
df=df[0:df["Open"].size-window]
df["target"]=target
df=df.drop(["Close_time","Taker_buy_base_asset_volume","Volume","Close","Low","High","Number_of_trades"],axis=1)
for i in range(1,4):
    df["mean_"+str(i*8)+"_hours"]=moving_average(df["Open"],i*3,4)
for i in range(1,3):
    df["mean_"+str(i*8)+"_days"]=moving_average(df["Open"],i*6*3,16)

In [5]:
drop_col=["Open",'Open_time','weekday']
open_delta=[]
for col in df.drop(drop_col,axis=1).columns:
    for i in range(df["Open"].size):
        df[col][i]=df[col][i]/df["Open"][i]
for i in range(1,df["Open"].size):
    df["Open_time"][i]/=86400
    #df["weekday"][i]/=6
    delta=df["Open"][i]-df["Open"][i-1]
    open_delta.append(delta/df["Open"][i])
df=df.drop(0,axis=0)

C:\Users\DANIEL~1\AppData\Local\Temp/ipykernel_14284/3722181041.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][i]=df[col][i]/df["Open"][i]
C:\Users\DANIEL~1\AppData\Local\Temp/ipykernel_14284/3722181041.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Open_time"][i]/=86400


In [6]:
df

,Open_time,Open,weekday,target,mean_8_hours,mean_16_hours,mean_24_hours,mean_8_days,mean_16_days
1,0.927083,1948.28,2,1.005225,1.006883,1.006883,1.006883,1.006883,1.006883
2,0.930556,1966.87,2,0.989440,0.997366,0.997366,0.997366,0.997366,0.997366
3,0.934028,1958.46,2,0.994812,1.001649,1.001649,1.001649,1.001649,1.001649
4,0.937500,1946.10,2,1.002605,1.008011,1.008011,1.008011,1.008011,1.008011
5,0.940972,1948.30,2,1.002294,1.006873,1.006873,1.006873,1.006873,1.006873
...,...,...,...,...,...,...,...,...,...
85034,0.451389,2059.36,4,0.994566,1.017334,1.021593,1.019999,0.954184,1.008234
85035,0.454861,2057.97,4,1.001725,1.014561,1.019675,1.021442,0.954828,1.008915
85036,0.458333,2048.17,4,1.005258,1.019416,1.024554,1.026329,0.959397,1.013742
85037,0.461806,2061.52,4,1.001014,1.012814,1.017919,1.019683,0.953184,1.007177


In [7]:
set(df["weekday"])

{0, 1, 2, 3, 4, 5, 6}

In [9]:
#for i in cols:
#    print(i)

In [10]:
cols=df.columns[3:]
scaling_range=0.2
for i in cols:
    for j in range(1,df["Open"].size):
        df[i][j]=(df[i][j]-(1-scaling_range))/(scaling_range*2)

C:\Users\DANIEL~1\AppData\Local\Temp/ipykernel_14284/3824140722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i][j]=(df[i][j]-(1-scaling_range))/(scaling_range*2)


In [11]:
window_size=30
X,y=df_to_X_y(df,window_size)
del(df)


In [12]:
opens=X[:,:,1][:,window_size-1]
X=np.concatenate([X[:,:,2:],X[:,:,[0]]],axis=2)
splitting=0.1
all_size=opens.shape[0]
opens_train=opens[:int(all_size*(1-splitting))]
opens_test=opens[int(all_size*(1-splitting)):]

In [13]:
opens_test.shape

(8501,)

In [31]:
with tf.device('/device:GPU:0'):
    X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                        test_size = 0.10,
                                                        random_state =123,
                                                        shuffle=False)

In [15]:

X_test[:,:,1][:,window_size-1].shape

(8501,)

In [36]:
with tf.device('/device:GPU:0'):
    opt = keras.optimizers.SGD(learning_rate=0.004,momentum=0.9)
    model = keras.Sequential()

    model.add(


    keras.layers.LSTM(

      units=128,

      input_shape=(X_train.shape[1], X_train.shape[2])

    )

  )

    model.add(keras.layers.Dense(units=64,activation='sigmoid'))

    model.add(keras.layers.Dropout(rate=0.7))

    model.add(keras.layers.Dense(units=64,activation='sigmoid'))

    model.add(keras.layers.Dense(units=1,use_bias=False))

    model.compile(loss='mean_absolute_error', optimizer=opt)

In [40]:
with tf.device('/device:GPU:0'):
    history = model.fit(

    X_train, y_train,

    epochs=50,

    batch_size=20,

    validation_split=0.1,

    shuffle=False

)

Epoch 1/50
3443/3443 [==============================] - 29s 8ms/step - loss: 0.0179 - val_loss: 0.0064
Epoch 2/50
3443/3443 [==============================] - 29s 9ms/step - loss: 0.0155 - val_loss: 0.0053
Epoch 3/50
3443/3443 [==============================] - 30s 9ms/step - loss: 0.0141 - val_loss: 0.0102
Epoch 4/50
3443/3443 [==============================] - 30s 9ms/step - loss: 0.0135 - val_loss: 0.0078
Epoch 5/50
3443/3443 [==============================] - 30s 9ms/step - loss: 0.0124 - val_loss: 0.0048
Epoch 6/50
3443/3443 [==============================] - 30s 9ms/step - loss: 0.0114 - val_loss: 0.0164
Epoch 7/50
3443/3443 [==============================] - 31s 9ms/step - loss: 0.0111 - val_loss: 0.0046
Epoch 8/50
3443/3443 [==============================] - 31s 9ms/step - loss: 0.0109 - val_loss: 0.0068
Epoch 9/50
3443/3443 [==============================] - 30s 9ms/step - loss: 0.0105 - val_loss: 0.0050
Epoch 10/50
3443/3443 [==============================] - 30s 9ms/step - l

KeyboardInterrupt: 

In [33]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend();



NameError: name 'history' is not defined

In [41]:
predictions=model.predict(X_test)

In [44]:
predictions

array([[0.48991027],
       [0.48991022],
       [0.48991013],
       ...,
       [0.48978832],
       [0.48978806],
       [0.48978817]], dtype=float32)

In [43]:
pd.DataFrame(predictions).describe()

,0
count,8501.000000
mean,0.489856
std,0.000115
min,0.489720
25%,0.489762
50%,0.489838
75%,0.489970
max,0.490112


In [ ]:
ьщвуд

In [ ]:
y_test

In [ ]:
#model.weights

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [21]:
import tensorflow as tf


config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

AttributeError: module 'tensorflow' has no attribute 'ConfigProto'

In [99]:
tf.debugging.set_log_device_placement(True)
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
